![Image](images/actinia_logo.png)

## Oberflächenklassifikation aus Luft- und Satellitenbildern mithilfe von actinia

### Die actinia Prozesskette

Actinia nutzt den **Prozesskettenansatz**, um für Import, Verarbeitung und
Export von Geodaten mit dem actinia GRASS GIS Verarbeitungssystem zu 
kommunizieren. Die Prozesskette muss in JSON formuliert sein.

Eine Prozesskette ist eine Liste von GRASS GIS Modulen, die nacheinander in der Reihenfolge der Liste ausgeführt werden. GRASS GIS Module werden als Prozessdefinitionen spezifiziert, die den Namen des Befehls, die Ein- und Ausgaben, einschließlich Import- und Exportdefinitionen sowie die Modulflags enthalten.

Dieser Workshop benutzt die Python Bibliothek **requests** für Interaktion mit dem actinia REST Dienst.

---
### actinia API Dokumentation

* [Stable actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/api/v3/swagger.json)
* [Development actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia-dev.mundialis.de/api/v3/swagger.json)

---
### Anforderungen

#### Software & Module

Dieses Tutorial setzt voraus, dass Sie mit der Programmiersprache [Python](https://python.org) vertraut sind. Die Kenntnis grundlegender [REST](https://de.wikipedia.org/wiki/Representational_State_Transfer)-API-Konzepte und deren Verwendung wird ebenfalls vorausgesetzt.

Die in diesem Tutorium verwendeten Python-Module sind:
* [requests](http://docs.python-requests.org/)
* [json](https://docs.python.org/3/library/json.html)
* [leafmap](https://leafmap.org/)
* [actinia-python-client](https://actinia-org.github.io/actinia-python-client/)


#### ACTINIA API Benutzer und Passwort

Für diesen Workshop werden die Anmeldedaten für die Authentifizierung benötigt, die unten in **Vorbereitung** als Variable festgelegt sind. Eine andere actinia-Instanz kann andere Anmeldedaten erfordern.

### Python Module und Hilfs-Funktionen

Bevor wir mit dem actinia-Server über Python interagieren, werden wir die erforderlichen Pakete importieren und eine Hilfsfunktion einrichten, um ein formatiertes JSON ausgeben zu lassen.

## Vorbereitung

Wir nutzen [Leafmap](https://leafmap.org) um die Ergebnisse von actinia zu visualisieren. Leafmap ist ein Python-Paket für interaktives Mapping und raumbezogene Analysen mit minimalem Programmieraufwand in einer Jupyter-Umgebung (siehe auch: [actinia with leafmap notebook](https://github.com/actinia-org/actinia-python-client/blob/main/notebooks/actinia_leafmap.ipynb)).

Als weiteres Paket installieren wir den Python Client Library für actinia ([actinia-python-client](https://github.com/actinia-org/actinia-python-client/)).

**Wichtig:** Danach müssen wir uns einmal ausloggen (im Menü File -> Log Out) und wieder einloggen.

In [ ]:
!pip install -U leafmap
!pip install actinia-python-client==0.4.1

Bitte einmal ausloggen (im Menü File -> Log Out) und wieder einloggen.

Im mundialis JupyterHub müssen danach unter Umständen Pfade neu gesetzt werden, damit matplotlib auch gefunden wird. Eventuelle Warnungen von pip install müssen berücksichtigt werden.

In [ ]:
import os
# '/home/jovyan/.local/bin'
path = os.getenv("PATH")
path = path + os.pathsep + "/home/jovyan/.local/bin"
os.environ["PATH"] = path
# adjust PYTHONPATH
import sys
sys.path.append("/home/jovyan/.local/lib/python3.11/site-packages")

In [ ]:
# Zuerst werden die erforderlichen Bibliotheken importiert.

import json

import leafmap
import requests
from requests.auth import HTTPBasicAuth
from actinia.utils import create_actinia_pc_item

Um die Kommunikation mit dem actinia Server zu vereinfachen, speichern wir die Anmeldedaten und die URL des REST-Servers in Variablen:

In [ ]:
# variables to set the actinia host, version, and user

actinia_baseurl = "https://actinia.mundialis.de"
actinia_version = "v3"
actinia_user = "fossgis2024"
actinia_pw = "ahd5ohghaitiiyoh0hu6"
actinia_url = f"{actinia_baseurl}/api/{actinia_version}"
# user, pw for the FOSSGIS 2023 workshop
actinia_auth = HTTPBasicAuth(actinia_user, actinia_pw)

## Hilfs-Funktionen

In [ ]:
# helper function to print formatted JSON using the json module

def print_as_json(data):
    print(json.dumps(data, indent=2))

# helper function to verify a request
def verify_request(request, success_code=200):
    if request.status_code != success_code:
        print("ERROR: actinia processing failed with status code %d!" % request.status_code)
        print("See errors below:")
        print_as_json(request.json())
        request_url = request.json()["urls"]["status"]
        requests.delete(url=request_url, auth=actinia_auth)
        raise Exception("The resource <%s> has been terminated." % request_url)

## Oberflächenklassifikation von Luftbildern

### Eingabedaten

In diesem Beispiel werden Digitale Orthophotos (DOPs) und ein normalisiertes digitales Oberflächenmodell (nDOM) als Input für eine Klassifikation verwendet.

DOPs sind Luftbildaufnahmen, die nicht nur georeferenziert sind, sondern auch mithilfe eines Oberflächenmodelles verfeinert wurden.

![Image](images/DOP.jpg)

Ein nDOM ist die Differenz zwischen einem Oberflächenmodell, das auch die absoluten Höhen von Objekten wie Bäumen und Gebäuden enthält, und einem Geländemodell, das die absolute Höhe des tatsächlichen Bodens angibt, auf dem die Objekte stehen.

![Image](images/nDOM.png)

### Ziel der Klassifikation

Diese Klassen sollen erkannt werden:

![Image](images/Legende_Klassen.png)



## Machine Learning

In der scikit Dokumentation für [supervised learning](https://scikit-learn.org/stable/supervised_learning.html) gibt es für jede Machine Learning Methode "Tips on Practical Use", die sehr hilfreich sind, um ein Modell zu optimieren.

### LogisticRegression

[scikit logistic regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

 * modelliert die Wahrscheinlichkeit der Zugehörigkeit zu einer bestimmten Klasse mit Hilfe der Sigmoidfunktion
 * In der Regel wird ein Schwellenwert festgelegt, der angibt, bei welchem Wert das Beispiel in die eine Klasse bzw. in die andere Klasse eingeordnet wird (binäre Klassifizierung).
 * Multi-Klassen-Klassifikation: in scitkit z.B. One-versus-rest (für N Klassen, werden N binäre Klassifizierer trainiert, und die Klasse mit der höchsten Wahrscheinlichkeit wird dem Beispiel zugeordnet)


### SVC

[Tips on practical use](https://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use)

 * Idee: Selektiere die Hyperebene, die den Abstand zwischen 2 Klassen maximiert
 * (Kernel-Trick: Bewegen zu anderen hoch Dimensionierten Raum, damit Daten nicht Linear trennbar sein müssen)
 * `r.learn.ml2`: nur lineare Kernel (GRASS GIS Addon [r.learn.ml2](https://grass.osgeo.org/grass-stable/manuals/addons/r.learn.ml2.html))
 * Multi-Klassen-Klassifikation: in scitkit z.B. One-vs-one (für N Klassen gibt es Nx(N-1)/2 Klassifikatoren: jede Klasse gegen jede)


![Image](images/iris_svc_001.png)

### Decision trees

[Tips on practical use](https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use)

 * Idee: Folge von Spaltungen des Input-raums definiert Region, die der Klasse entspricht
 * divide-and-conquer Strategie
 * &plus; leicht zu verstehen
 * &plus; hoch dimensionierte Daten + große Datenmengen
 * &minus; optimalen Split finden NP-hart
 * &minus; Overfitting

#### RandomForestClassifier

 * Bagging mit Decision Trees (DT) = Kombination von mehreren DT, die auf unterschiedliche Subsamples trainiert werden um einen besseren Klassifikator zu erlangen
 * &plus; reduziert Overfitting
 * &plus; leicht parallelisierbar

![Image](images/iris_decision_trees.svg)

###  KNeighborsClassifier

[scikit neighbors](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification)

 * Verteilung der Features durch Trainingsdaten modelliert
 * Klasse gewählt basierend auf nächsten Feature in Trainingsdaten
 * &plus; Trainingsdaten repräsentieren die Verteilung der Features
 * &plus; robuster als Nearest Neighbor
 * &minus; Rauschen, wenig robust gegen Ausreißer (vor allem wenn k klein gewählt ist)
 * &minus; viele Trainingsdaten benötigt
 * k meist 2-10
 * Features sollten gleich skaliert sein

### MLPClassifier

[Tips on practical use](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#tips-on-practical-use)

 * Multilayer perceptron
 * einfaches Neuronales Netz (fully connected, feed-forward)
 * Paramter zum Einstellen:
   * Anzahl der Layer + Neuronen pro Layer
   * Aktivierungsfunktion (Zusammenhang zwischen den Neuronen)
   * Lernrate (wie stark Gewichte angepasst werden)
   * Optimierungsalgorithmus (Lossfunction, ...)
 * &minus; viele Trainingsdaten
 * &minus;/&plus; viele Parameter 

### GaussianNB

[scikit Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes)

 * basiert auf "Bayes Theorem": Diese Klassifikatoren gehen davon aus, dass der Wert eines bestimmten Merkmals unabhängig vom Wert eines anderen Merkmals ist
 * ordnet jedes Objekt der Klasse zu, zu der es mit der größten Wahrscheinlichkeit gehört
 * Wahrscheinlichkeit eines Features ist durch Gaussche Verteilung repräsentiert
 * &plus; benötigt nur wenig Trainingsdaten, um die für die Klassifizierung erforderlichen Parameter zu schätzen + haben einfachen Aufbau


### Erstellen einer Prozesskette Schritt für Schritt

Zuerst wird die actinia request URL für asynchrones Prozessieren gesetzt und eine leere Prozesskette erstellt.

In [ ]:
# create a POST request to the Actinia Data API
request_url = f"{actinia_url}/locations/fossgis2024_epsg25832_utm32N/processing_async_export"

process_chain = {"version": 1, "list": []}

### Hinzufügen von Einträgen zu der Prozeskette

Setzen der aktuellen Region für Raster-Prozessierung

In [ ]:
# list item for g.region
list_id = "g_region_to_dop"
# long form, as in a process chain
inputs = [
              {
                  "param": "raster",
                  "value": "dop_nir@PERMANENT"
              }
          ]

# short form, accepted by create_actinia_pc_item
inputs = {"raster": "dop_nir@PERMANENT"}

flags = "p"
stdout = {"id": "region_to_dop", "format": "list", "delimiter": "\n"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="g.region",
                                 inputs=inputs,
                                 stdout=stdout,
                                 flags=flags)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### NDVI berechnen

In [ ]:
# list item for r.mapcalc
list_id = "r_mapcalc_ndvi"
# long form
inputs = [
              {
                  "param": "expression",
                  "value": "ndvi = int(127.5 * ( float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT)) + 1.0 ) )"
              }
          ]

# short form
inputs = {"expression": "ndvi = int(127.5 * ( float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT)) + 1.0 ) )"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.mapcalc",
                                 inputs=inputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Info zum NDVI Raster

In [ ]:
# list item for r.info
list_id = "r_info_ndvi"
# long form
inputs = [
              {
                  "param": "map",
                  "value": "ndvi"
              }
          ]

# short form
inputs = {"map": "ndvi"}
flags = "r"

stdout = {"id": "ndvi_range", "format": "kv", "delimiter": "="}
pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.info",
                                 inputs=inputs,
                                 flags=flags,
                                 stdout=stdout)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Gruppe erstellen

In [ ]:
# list item for i.group
list_id = "create_group"
# long form
inputs = [{"param": "input",
           "value": "dop_red@PERMANENT,dop_green@PERMANENT,dop_blue@PERMANENT,dop_nir@PERMANENT,ndvi,ndom@PERMANENT"
           }]
outputs = [{"param": "group",
            "value": "classification_input"
           }]
# short form
inputs = {"input": "dop_red@PERMANENT,dop_green@PERMANENT,dop_blue@PERMANENT,dop_nir@PERMANENT,ndvi,ndom@PERMANENT"}
outputs = {"group": "classification_input"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="i.group",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Trainings- und Validierungsdaten erstellen

In [ ]:
# list item for v.divide.training_validation
list_id = "split_training_validation_data"
# long form
inputs = [{"param": "input",
           "value": "training_set@PERMANENT"
          },
          {"param": "column",
           "value": "classnum"
          },
          {"param": "training_percent",
           "value": "70"
          }]
outputs = [{"param": "training",
            "value": "tr_data"
           },
           {"param": "validation",
            "value": "val_data"
           }]
# short form
inputs = {"input": "training_set@PERMANENT",
          "column": "classnum",
          "training_percent": "70"}

outputs = {"training": "tr_data",
           "validation": "val_data"
          }

pc_item = create_actinia_pc_item(id=list_id,
                                 module="v.divide.training_validation",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Trainieren des ML Modelles mit `r.learn.train`

Hier am Beispiel eines Random Forest Classifiers, siehe Dokumentation für [r.learn.train](https://grass.osgeo.org/grass82/manuals/addons/r.learn.train.html), z.B. welche Methoden verfügbar sind.

In [ ]:
# list item for r.learn.train
list_id = "train_classifier"
# long form
inputs = [{"param": "group",
           "value": "classification_input"
          },
          {"param": "training_points",
           "value": "tr_data"
          },
          {"param": "cv",
           "value": "5"
          },
          {"param": "model_name",
           "value": "RandomForestClassifier"
          },
          {"param": "field",
           "value": "classnum"
          }]
outputs = [{"param": "save_model",
            "value": "RF_model.gz"
           }]

# short form
inputs = {"group": "classification_input",
          "training_points": "tr_data",
          "cv": "5",
          "model_name": "RandomForestClassifier",
          "field": "classnum"
         }
outputs = {"save_model": "RF_model.gz"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.learn.train",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Eigentliche Klassifikation

Anwenden des trainierten Modelles mit `r.learn.predict`

In [ ]:
# list item for r.learn.predict
list_id = "apply_classifier"
# long form
inputs = [{"param": "group",
           "value": "classification_input"
          },
          {"param": "load_model",
           "value": "RF_model.gz"
          }]
outputs = [{"param": "output",
            "value": "classification_RF"
           }]

# short form
inputs = {"group": "classification_input",
          "load_model": "RF_model.gz"}
outputs = {"output": "classification_RF"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.learn.predict",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Validierung

In [ ]:
# list item for r.confusionmatrix
list_id = "validation"
# long form
inputs = [{"param": "classification",
           "value": "classification_RF"
          },
          {"param": "vector_reference",
           "value": "val_data"
          },
          {"param": "column",
           "value": "classnum"
          }]
# short form
inputs = {"classification": "classification_RF",
          "vector_reference": "val_data",
          "column": "classnum"}

stdout = {"id": "validation", "format": "list", "delimiter": "\n"}
flags = "m"
pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.confusionmatrix",
                                 inputs=inputs,
                                 stdout=stdout,
                                 flags=flags)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Ergebnisse exportieren

In [ ]:
# list item for the exporter
list_id = "export_classification"
# long form
outputs = [{"export": {"type": "raster", "format": "GTiff"},
            "param": "map",
            "value": "classification_RF"}]

# short form not working here because of the special "export" entry

pc_item = create_actinia_pc_item(id=list_id,
                                 module="exporter",
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

#### Job submission

Die Prozesskette wird mit der POST Methode an actinia geschickt

In [ ]:
# submit the job
request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)
# check if anything went wrong
verify_request(request, 200)

#### actinia Antwort

actinia schickt eine Antwort zurück mit Informationen, ob die Prozesskette akzeptiert wurde

In [ ]:
# get a json-encoded content of the response
jsonResponse = request.json()

print(f"Response with status code: {request.status_code}")

# print formatted JSON
print_as_json(jsonResponse)

#### Job Status

Der Status des actinia jobs wird abgefragt

In [ ]:
# make a GET request to the Actinia Data API
request_url = jsonResponse["urls"]["status"]
print("actinia GET request:")
print(request_url)
print("---")

request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

#### Job Ende

Der Status des actinia jobs wird weiter abgefragt, bis er erfolgreich beendet wurde oder auf einen Fehler läuft. Wenn der Job erfolgreich beendet wurde, steht in der actinia Antwort "Processing successfully finished"

In [ ]:
# continue polling until finished
while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)

#### Job logs und Ergebnisse

actinia Antwort anzeigen

In [ ]:
# print formatted JSON
print_as_json(jsonResponse)

Ergebnis-URL aus actinia Antwort selektieren und anzeigen.

In [ ]:
result_url = request.json()["urls"]["resources"][0]

print(result_url)
raster_url = result_url.replace("//", f"//{actinia_user}:{actinia_pw}@")

### Ergebnis hier im notebook anzeigen

In [ ]:
# visualization with leafmap
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_cog_layer(
    raster_url,
    label="Klassifikation RF",
    colormap_name="set1",
)
legend_dict = {
    "1 Gebäude": "e41a1c",
    "2 Wasser": "377eb8",
    "3 Niedrige Vegetation": "4daf4a",
    "4 Hohe Vegetation": "984ea3",
    "5 Versiegelte Fläche": "ff7f00",
    "6 Kahler Boden": "ffff33",
}
m.add_legend(
    title="Klassifikation", legend_dict=legend_dict, draggable=False
)
# show map
m

Prozessergebnisse ausgeben lassen:

In [ ]:
print("Process results:")
print_as_json(jsonResponse["process_results"])

## Eigene Klassifikation

Im Folgenden kann mit verschiedenen Input-Daten und/oder ML-Methoden experimentiert werden.\
Als Erstes muss wieder die region gesetzt, sowie der NDVI berechnet werden. Hier muss nichts geändert werden:

In [ ]:
process_chain = {"version": 1, "list": []}

# short form, accepted by create_actinia_pc_item
inputs = {"raster": "dop_nir@PERMANENT"}

flags = "p"
stdout = {"id": "region_to_dop", "format": "list", "delimiter": "\n"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="g.region",
                                 inputs=inputs,
                                 stdout=stdout,
                                 flags=flags)
process_chain["list"].append(pc_item)

# short form
inputs = {"expression": "ndvi = int(127.5 * ( float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT)) + 1.0 ) )"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.mapcalc",
                                 inputs=inputs)
process_chain["list"].append(pc_item)

Nun wird der Input für die Klassifikation definiert:
Über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers` können die potenziellen Input-Daten angezeigt werden 

In [ ]:
# make a GET request to the actinia data API
request_url = f"{actinia_url}/locations/fossgis2024_epsg25832_utm32N/mapsets/PERMANENT/raster_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster layers in mapset PERMANENT of location fossgis2024_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])


Im Folgenden können nach Belieben die Input Daten variiert werden.\
*Hinweis:* Sie müssen den Namen der Karte und des Mapsets folgendermaßen angeben: "map_name@mapset_name".

In [ ]:
# list item for i.group
list_id = "create_group"

# TODO: Input setzen
inputs = {"input": "map_name@PERMANENT,..."}

outputs = {"group": "classification_input"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="i.group",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

Der Input wird wieder in ein Trainings und Validierungsset aufgeteilt. Hier muss nichts geändert werden.\
Optional kann der Anteil an Trainingsdaten variiert werden (aktuell bei 70%)

In [ ]:
# list item for v.divide.training_validation
list_id = "split_training_validation_data"

# short form
inputs = {"input": "training_set@PERMANENT",
          "column": "classnum",
          "training_percent": "70"}

outputs = {"training": "tr_data",
           "validation": "val_data"
          }

pc_item = create_actinia_pc_item(id=list_id,
                                 module="v.divide.training_validation",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

Nun wird die ML-Methodik gewählt (Dokumentation möglicher Methodiken und Parameter siehe [r.learn.train](https://grass.osgeo.org/grass82/manuals/addons/r.learn.train.html))

In [ ]:
# list item for r.learn.train
list_id = "train_classifier"

inputs = {"group": "classification_input",
          "training_points": "tr_data",
          "field": "classnum",
          # TODO: Auswahl einer ML-Methodik, sowie setzen verschiedener Parameter
         }
# TODO: Model-name vergeben
outputs = {"save_model": "<modelname>.gz"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.learn.train",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

Anwenden des trainierten Modelles mit `r.learn.predict`

In [ ]:
# list item for r.learn.predict
list_id = "apply_classifier"


# TODO: Modelname und output definieren
inputs = {"group": "classification_input",
          "load_model": "<modelname>.gz"}
outputs = {"output": "<outputname>"}

pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.learn.predict",
                                 inputs=inputs,
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

Validierung der Ergebnisse:

In [ ]:
# list item for r.confusionmatrix
list_id = "validation"

# TODO: output aus vorigem Schritt angeben
inputs = {"classification": "<outputname>",
          "vector_reference": "val_data",
          "column": "classnum"}

stdout = {"id": "validation", "format": "list", "delimiter": "\n"}
flags = "m"
pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.confusionmatrix",
                                 inputs=inputs,
                                 stdout=stdout,
                                 flags=flags)
process_chain["list"].append(pc_item)

Ausgeben der Prozesskette

In [ ]:
print_as_json(process_chain)

Ergebnisse exportieren

In [ ]:
# list item for the exporter
list_id = "export_classification"
# long form
outputs = [{"export": {"type": "raster", "format": "GTiff"},
            "param": "map",
            "value": "<outputname>"}]

# short form not working here because of the special "export" entry

pc_item = create_actinia_pc_item(id=list_id,
                                 module="exporter",
                                 outputs=outputs)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

Nun wird die Prozesskette an actinia geschickt und prozessiert:

In [ ]:
# submit the job
request_url = f"{actinia_url}/locations/fossgis2024_epsg25832_utm32N/processing_async_export"
request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)
# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print(f"Response with status code: {request.status_code}")

# print formatted JSON
print_as_json(jsonResponse)

# make a GET request to the Actinia Data API
request_url = jsonResponse["urls"]["status"]
print("actinia GET request:")
print(request_url)
print("---")

# continue polling until finished
while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

Visualisierung der Ergebnisse:

Ergebnisse anzeigen, die URL ist in der Antwort.

In [ ]:
result_url = request.json()["urls"]["resources"][0]

print(result_url)

In [ ]:
raster_url = result_url.replace("//", f"//{actinia_user}:{actinia_pw}@")

# visualization with leafmap
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_cog_layer(
    raster_url,
    label="Eigene Klassifikation",
    colormap_name="set1",
)
m.add_legend(
    title="Klassifikation", legend_dict=legend_dict, draggable=False# , style=style
)
# show map
m


Prozessergebnisse ausgeben lassen:

In [ ]:
print("Process results:")
print_as_json(jsonResponse["process_results"])